In [1]:
import torch
import torch.nn as nn
import numpy as np


In [39]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [40]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()
        self.conv = nn.Conv1d(in_features, in_features, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm1d(in_features)
        
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = torch.relu(out + x)
        return out

class ResNet(nn.Module):
    def __init__(self, in_features, out_features, num_residual_blocks):
        super(ResNet, self).__init__()
        self.conv = nn.Conv1d(in_features, out_features, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm1d(out_features)
        self.residual_blocks = nn.Sequential(*[ResidualBlock(out_features) for _ in range(num_residual_blocks)])
        self.fc = nn.Linear(out_features, 1)
        
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.conv(x)
        out = self.bn(out)
        out = torch.relu(out)
        
        out = self.residual_blocks(out)
        out = torch.mean(out, dim=2)
        out = self.fc(out)
        return out

In [41]:
# Import necessary libraries
import torch
import numpy as np

# Generate some fake data
np.random.seed(42)
input_data = np.random.normal(0, 1, (100, 3, 50))
target_data = np.random.normal(0, 1, (100, 1))

# Convert data to tensors
input_tensor = torch.tensor(input_data, dtype=torch.float32)
target_tensor = torch.tensor(target_data, dtype=torch.float32)

# Create model, criterion and optimizer
model = ResNet(in_features=3, out_features=32, num_residual_blocks=5)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(input_tensor)
    loss = criterion(outputs, target_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/100], Loss: {loss.item():.4f}')


Epoch [10/100], Loss: 0.6652
Epoch [20/100], Loss: 0.2800
Epoch [30/100], Loss: 0.0416
Epoch [40/100], Loss: 0.0086
Epoch [50/100], Loss: 0.0035
Epoch [60/100], Loss: 0.0008
Epoch [70/100], Loss: 0.0004
Epoch [80/100], Loss: 0.0001
Epoch [90/100], Loss: 0.0000
Epoch [100/100], Loss: 0.0000


In [42]:
print(f'Number of parameters: {count_parameters(model)}')

Number of parameters: 16257


In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv1d(in_features, in_features, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm1d(in_features)
        self.conv2 = nn.Conv1d(in_features, in_features, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm1d(in_features)

    def forward(self, x):
        residual = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += residual
        out = F.relu(out)
        return out

class ResNet4(nn.Module):
    def __init__(self, in_features, num_classes):
        super(ResNet4, self).__init__()

        self.conv1 = nn.Conv1d(in_features, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm1d(64)
        self.layer1 = self._make_layer(64, 2)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, out_features, num_blocks, stride=1):
        layers = []
        for _ in range(num_blocks):
            layers.append(ResidualBlock(out_features))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = torch.mean(out, dim=-1)
        out = self.fc(out)
        return out


In [45]:
import torch
import numpy as np

# Dummy 1D regression data
num_samples = 128
num_features = 32
inputs = torch.tensor(np.random.randn(num_samples, num_features), dtype=torch.float32)
targets = torch.tensor(np.random.randn(num_samples, 1), dtype=torch.float32)

# Initialize the model, loss function, and optimizer
model = ResNet4(in_features=inputs.shape[1], num_classes=1)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

# Train the model
for epoch in range(100):
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/100], Loss: {loss.item()}')

# Evaluate the model on test data
with torch.no_grad():
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    print(f'Test Loss: {loss.item()}')


RuntimeError: Given groups=1, weight of size [64, 32, 3], expected input[1, 128, 32] to have 32 channels, but got 128 channels instead

In [51]:
inputs.shape, outputs.shape

(torch.Size([128, 32]), torch.Size([100, 1]))